In [1]:
# import ipyparallel as ipp
from multiprocessing import Pool
# from multiprocessing import Queue
# from multiprocessing import Process

import EEGAnalysis as ea
import h5py
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt
import re, os
from tqdm import tqdm

In [11]:
dataset = ea.DataManager('/home/yizhan/storage/EEG/Data')

patient_id = 'zhouchen'
patient = dataset.get_patient(patient_id)

# contants
_freq = 2000
ROI = (-3, 3)
tspec = np.linspace(ROI[0],ROI[1],_freq * (ROI[1] - ROI[0]))

nyq = _freq / 2.0
width = 5.0 / nyq
ripple_db = 60.0

# Compute the order and Kaiser parameter for the FIR filter.
N, beta = signal.kaiserord(ripple_db, width)

taps = signal.firwin(N, [0.5/nyq, 2/nyq],window=('kaiser', beta),pass_zero=False)

In [12]:
# preview date and paradigm
chidx = 27
data_list = patient.load_isplit(chidx)
list(data_list.keys())

['180827-1',
 '180827-2',
 '180827-3',
 '180829-1',
 '180829-2',
 '180829-3',
 '180831-1',
 '180831-2',
 '180831-3',
 '180901-1',
 '180901-2',
 '180901-3',
 '180902-1',
 '180902-2',
 '180902-3',
 '180903-1',
 '180903-2',
 '180903-3',
 '180904-1',
 '180904-2',
 '180904-3']

In [4]:
def _load_phase(inputs):
    chidx, idate = inputs
    data_list = patient.load_isplit(chidx, idate)

    _entry = data_list[idate]['value']
    # _marker = patient.get_marker(file=idate, paradigm='%s-10'%idate[-1])
    _marker = patient.get_marker(file=idate)[20:]

    try:
        _filted = signal.hilbert(signal.filtfilt(taps, 1, _entry))
    except ValueError:
        print('skip channel %03d, %s'%(chidx, idate))
        return {'%s/%s/%03d'%(_date_str, _pardigm_str, chidx): None}
        
    _epoch = ea.create_1d_epoch_bymarker(_filted, _marker, ROI, int(_freq))

    _date_str, _pardigm_str = re.findall(r'(\d{6})-(\d{1})', idate)[0]
    return {'%s/%s/%03d'%(_date_str, _pardigm_str, chidx): _epoch}

In [ ]:
_inputs = []
_total_ch = len(patient._sgch_config['chidx'])

for chidx in range(_total_ch):
    for idate in data_list:
        _inputs.append((chidx, idate))
        
with Pool(processes=5) as _p:
    result = list(tqdm(_p.imap_unordered(_load_phase, _inputs), total=len(_inputs)))

 13%|█▎        | 305/2331 [03:57<20:55,  1.61it/s]

In [8]:
# export hdf5
_data_export = h5py.File('%s-5-delta.h5'%patient_id, 'w')

# setup hdf5 groups
for idate in np.unique([idate[:6] for idate in data_list.keys()]):
    [_data_export.create_group('%s/%d'%(idate, i)) for i in range(1,4)]

for item in tqdm(result):
    for _key, _val in item.items():
        _data_export.create_dataset(_key, data=_val, compression=4)

_data_export.close()
result = None  # clear memory

100%|██████████| 1743/1743 [04:04<00:00,  7.30it/s]


---

linear step

In [6]:
_data_export = h5py.File('%s-5-delta.h5'%patient_id, 'w')

# setup hdf5 groups
for idate in np.unique([idate[:6] for idate in data_list.keys()]):
    [_data_export.create_group('%s/%d'%(idate, i)) for i in range(1,4)]


_total_ch = len(patient._sgch_config['chidx'])

pbar = tqdm(total=_total_ch*len(data_list))
for chidx in range(_total_ch):
    data_list = patient.load_isplit(chidx)
    for idate in data_list.keys():
        _entry = data_list[idate]['value']
#         _marker = patient.get_marker(file=idate, paradigm='%s-10'%idate[-1])
        _marker = patient.get_marker(file=idate)[20:]

        try:
            _filted = signal.hilbert(signal.filtfilt(taps, 1, _entry))
        except ValueError:
            print('skip channel %03d, %s'%(chidx, idate))
            continue
        _epoch = ea.create_1d_epoch_bymarker(_filted, _marker, ROI, int(_freq))

        _date_str, _pardigm_str = re.findall(r'(\d{6})-(\d{1})', idate)[0]
        _data_export.create_dataset('%s/%s/%03d'%(_date_str, _pardigm_str, chidx), data=_epoch, compression=4)
        pbar.update(1)
pbar.close()

_data_export.close()

100%|██████████| 1743/1743 [39:12<00:00,  1.11s/it] 


---

tapping locking

In [3]:
# _behavior = pd.read_csv('wuzhenwei-behavior-final.csv', dtype={'date':'str'})
# _behavior = patient.get_marker('zhangchen-behavior-final', dtype={'date':'str'})
# check_behavior_prediction = lambda x,y,z: _behavior.time[(_behavior.paradigm == x) & (_behavior['date'] == y)].values < z
# _behavior[_behavior.paradigm=='1-5'].head()

In [17]:
_data_export = h5py.File('%s-10-delta-tapping-lock.h5'%patient_id, 'w')

for idate in np.unique([idate[:6] for idate in data_list.keys()]):
    [_data_export.create_group('%s/%d'%(idate, i)) for i in range(1,4)]

chidx = 10
pbar = tqdm(total=110*7*3)
for chidx in range(110):
    data_list = patient.load_isplit(chidx)
    for idate in data_list.keys():
        if idate[-1] != '1':
            pbar.update(1)
            continue
        _entry = data_list[idate]['value']
        
        _t = patient.get_marker(idate, paradigm='%s-10'%idate[-1])
        _b = _behavior.time[(_behavior.date == idate[:6]) & (_behavior.paradigm == '%s-10'%idate[-1])].values - 0
        _marker = _t + _b

        _filted = signal.hilbert(signal.filtfilt(taps, 1, _entry))
        _epoch = ea.create_1d_epoch_bymarker(_filted, _marker, ROI, int(_freq))

        _date_str, _pardigm_str = re.findall(r'(\d{6})-(\d{1})', idate)[0]
        _data_export.create_dataset('%s/%s/%03d'%(_date_str, _pardigm_str, chidx), data=_epoch, compression=4)
        pbar.update(1)
pbar.close()

100%|██████████| 2310/2310 [49:53<00:00,  1.30s/it] 


In [14]:
_data_export.close()

In [15]:
pbar.close()